In [26]:
import platform
import os
import numpy as np
import tensorflow as tf
import keras
from keras import layers

gpu_token = tf.test.is_gpu_available('GPU')

if platform.machine() == 'x86_64':
    if gpu_token:
        os.chdir('/home/ucdasec/Faulthunter-RNN-approach')
    else:
        os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach")
elif platform.machine() == 'arm64':
    os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach")

from Utils.dataframe_gen import *
from Utils.OneHot_gen import *
from Utils.TFIDF_gen import *
from Utils.word2vec_gen import *
from Utils.vector_gen import *



2024-03-05 15:01:14.174675: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 15:01:14.174945: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 15:01:14.175098: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [27]:
# with tf.device('CPU'):
#     original_df = dataframe_init(gpu_token)
#     word2vec_df = original_df.copy()
#     word2vec_df = word2vec_init(word2vec_df, gpu_token)
#     vectors = vec_split(word2vec_df)
#     tensors = tensor_gen(vectors)

#     model = keras.Sequential()

#     model.add(layers.Input((218, 1)))
#     # model.add(layers.SimpleRNN(128, return_sequences=True, dropout=0.4))
#     # model.add(layers.SimpleRNN(128, return_sequences=True, dropout=0.4))
#     model.add(layers.SimpleRNN(128, dropout=0.4))
#     model.add(layers.Dense(128, activation=keras.activations.relu)) #kernel_regularizer=keras.regularizers.l2(0.01)
#     # model.add(layers.Dense(128, activation=keras.activations.relu ))
#     # model.add(layers.Dense(128, activation=keras.activations.relu ))
#     model.add(layers.Dense(2, activation = keras.activations.softmax))

#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True,gamma=2),
#                 metrics=["accuracy"], )
#     model.summary()


In [29]:
original_df = dataframe_init(gpu_token)
# word2vec_df = original_df.copy()
# word2vec_df = word2vec_init(word2vec_df, gpu_token)

tfidf_df = original_df.copy()
tfidf_df = TFIDF_init(tfidf_df)

/home/ucdasec/Faulthunter-RNN-approach/Utils/TFIDF_gen.py:25: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Lines'][vector[0]] = vector[1]


In [30]:
vectors = vec_split(tfidf_df)
tensors = tensor_gen(vectors)

In [35]:
model = keras.Sequential()

model.add(layers.Input((tensors[0].shape[1], 1)))
model.add(layers.SimpleRNN(1024, return_sequences=True, dropout=0.4))
model.add(layers.SimpleRNN(1024, return_sequences=True, dropout=0.4))
model.add(layers.SimpleRNN(1024, dropout=0.4))
model.add(layers.Dense(1024, activation=keras.activations.relu)) #kernel_regularizer=keras.regularizers.l2(0.01)
model.add(layers.Dense(1024, activation=keras.activations.relu ))
model.add(layers.Dense(1024, activation=keras.activations.relu ))
model.add(layers.Dense(2, activation = keras.activations.softmax))

model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0001), loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True,gamma=2),
              metrics=["accuracy"], )
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_14 (SimpleRNN)   (None, 2020, 1024)        1050624   
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 2020, 1024)        2098176   
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, 1024)              2098176   
                                                                 
 dense_20 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_21 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_22 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_23 (Dense)            (None, 2)                

In [37]:
# with tf.device('GPU'):
history = model.fit(tensors[0], tensors[2], epochs=50)

Epoch 1/50
131/131 [==============================] - 332s 3s/step - loss: 0.0116 - accuracy: 0.9810
Epoch 2/50
131/131 [==============================] - 332s 3s/step - loss: 0.0108 - accuracy: 0.9813
Epoch 3/50
131/131 [==============================] - 332s 3s/step - loss: 0.0107 - accuracy: 0.9813
Epoch 4/50
131/131 [==============================] - 332s 3s/step - loss: 0.0106 - accuracy: 0.9813
Epoch 5/50
131/131 [==============================] - 332s 3s/step - loss: 0.0107 - accuracy: 0.9813
Epoch 6/50
131/131 [==============================] - 332s 3s/step - loss: 0.0106 - accuracy: 0.9813
Epoch 7/50
131/131 [==============================] - 332s 3s/step - loss: 0.0101 - accuracy: 0.9813
Epoch 8/50
131/131 [==============================] - 332s 3s/step - loss: 0.0104 - accuracy: 0.9813
Epoch 9/50
131/131 [==============================] - 332s 3s/step - loss: 0.0106 - accuracy: 0.9813
Epoch 10/50
131/131 [==============================] - 332s 3s/step - loss: 0.0104 - accura

In [34]:
!nvidia-smi

Tue Mar  5 17:36:50 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 27%   31C    P8     5W / 151W |   7694MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
tensors[2].shape

TensorShape([4163, 2])

In [ ]:
y_pred = model.predict(x_test)

33/33 [==============================] - 3s 81ms/step


In [ ]:
list(y_pred)

[array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 

In [ ]:
(y_test[1040])

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>